http://web.eecs.umich.edu/~mihalcea/downloads.html

In [180]:
import nltk
from nltk.corpus import wordnet as wn
from nltk.corpus import semcor
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords 
import random
import string
from pprint import pprint

### Funzioni usate dall'algoritmo di Lesk

In [181]:
def get_context(sentence):
    # bag of words approach
    sentence.lower()
    tokens = word_tokenize(sentence) 
    tokens = list(filter(lambda token: token not in string.punctuation, tokens))
    wnl = nltk.WordNetLemmatizer()
    ps = PorterStemmer()
    return [ps.stem(wnl.lemmatize(token.lower())) for token in tokens if token not in stopwords.words('english')] #lemma o stemma? solo uno dei due da lasciare o no?

def get_gloss_examples(sense):
    gloss = get_context(sense.definition())
    examples = get_context(' '.join((sense).examples()))
    return gloss + examples

def get_overlap(list_1, list_2):
    return [value for value in list_1 if value in list_2]

### Algoritmo di Lesk

In [182]:
def lesk(word, sentence):
    best_sense = wn.synsets(word)[0]
    max_overlap = 0
    context = get_context(sentence)
    sifnature = None
    overlap = None
    for sense in wn.synsets(word):
        signature = get_gloss_examples(sense)
        overlap = get_overlap(context, signature)
        if len(overlap) > max_overlap:
            max_overlap = len(overlap)
            best_sense = sense
    return best_sense

In [ ]:
sentence = "i drink wine using a glass"
print(wn.synsets('glass')[1].definition())
lesk('glass', sentence)

### Funzioni utili per l'esecuzione dell'algoritmo di Lesk su 50 frasi scelte randomicamente da SemCor

In [377]:
def get_rand_elems(array):
    rands = random.sample(range(0, len(array)), 50)
    return [array[i] for i in rands]

def get_rand_elems_2(sentences):
    phrases = []
    extracted = []
    while len(phrases) < 50 :
        rand = random.randint(0, len(sentences))
        if '\'NN\'' in str(sentences[rand]) and rand not in extracted and '.n.' in str(sentences[rand]):
            phrases.append(sentences[rand])
            extracted.append(rand)
    return phrases, extracted

def get_nouns(list_of_sentences):
    nouns = []
    ps = PorterStemmer()
    wnl = WordNetLemmatizer()
    for sentence in list_of_sentences:
        for word in sentence:
            if word.pos()[0][1] == 'NN' and '.n.' in str(word):
                nouns.append([word.leaves(), word.label()])
            #prendo solo parole singole per evitare i noun phrase e che inizino con la minuscola così evito i nomi propri.
                #nouns.append([str(word).split('NN ')[1], str(word).split('\'')[1]])
                break
    return nouns    

     
def list_to_string(s):
    str1 = ''
    for ele in s:
        str1 += ele + ' '  
    return str1

### Esecuzione

Frasi annotatate con synset su SemCor

In [268]:
sentences = semcor.sents()
tagged_sentences = semcor.tagged_sents(tag="both")

Selezione randomica di 50 frasi contenenti almeno un sostantivo e indici delle frasi estratte

In [413]:
random_sentences, rand = get_rand_elems_2(tagged_sentences)

estrazione di sostantivi con relativi synset dalle frasi estratte

In [414]:
nouns = get_nouns(random_sentences)

esecuzione dell'algoritmo di Lesk su ogni sostantivo estratto

In [415]:
correct = 0
for i in range(len(nouns)):
    actual_lesk = lesk(nouns[i][0][0], list_to_string(sentences[rand[i]]))
    print(f"WORD: {nouns[i][0][0]} --> in SENTENCE: {list_to_string(sentences[rand[i]])}") 
    print(f"LESK SYNSET {actual_lesk} REAL SYNSET: {nouns[i][1]}")
    if str(actual_lesk).split('\'')[1] in str(nouns[i][1]):
        print("CORRECT\n\n\n")
        correct += 1
    else:
        print("WRONG\n\n\n")

WORD: definition --> in SENTENCE: When I pressed for a purely religious definition , I encountered the familiar blend of liberal piety , interfaith good will , and a small residue of ethnic loyalty . 
LESK SYNSET Synset('definition.n.02') REAL SYNSET: Lemma('definition.n.01.definition')
WRONG



WORD: Legislature --> in SENTENCE: It urged that the next Legislature `` provide enabling funds and re-set the effective date so that an orderly implementation of the law may be effected '' . 
LESK SYNSET Synset('legislature.n.01') REAL SYNSET: Lemma('legislature.n.01.legislature')
CORRECT



WORD: miles --> in SENTENCE: In 1602 , George Waymouth , in the same little Discovery that Hudson now commanded , had sailed 300 miles up the strait before his frightened men turned the ship back . 
LESK SYNSET Synset('mile.n.03') REAL SYNSET: mile.n.2;1
WRONG



WORD: techniques --> in SENTENCE: Applying the techniques developed at Harvard for generating a paradigm from a representative form and its class

In [416]:
print(f"CORRECT ANSWERS: {correct}")
print(f"TOTAL ANSWERS: {len(nouns)}")
print(f"ACCURACY: {correct/len(nouns)}")

CORRECT ANSWERS: 27
TOTAL ANSWERS: 50
ACCURACY: 0.54
